In [ ]:
import torch
import torch.nn as nn
from libs.models import encoder3,encoder4
from libs.models import decoder3,decoder4
import numpy as np
from libs.Matrix import MulLayer
from libs.Criterion import LossCriterion
from sklearn.decomposition import PCA


class LossCriterion(nn.Module):
    def __init__(self, style_layers, content_layers, style_weight, content_weight):
        super(LossCriterion, self).__init__()
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.style_weight = style_weight
        self.content_weight = content_weight
        self.styleLosses = [styleLoss()] * len(style_layers)
        self.contentLosses = [nn.MSELoss()] * len(content_layers)

    def forward(self, tF, sF, cF):
        # Content loss
        totalContentLoss = 0
        for i, layer in enumerate(self.content_layers):
            cf_i = cF[layer].detach()
            tf_i = tF[layer]
            loss_i = self.contentLosses[i]
            totalContentLoss += loss_i(tf_i, cf_i)
        totalContentLoss = totalContentLoss * self.content_weight

        # Style loss
        totalStyleLoss = 0
        for i, layer in enumerate(self.style_layers):
            sf_i = sF[layer].detach()
            tf_i = tF[layer]
            loss_i = self.styleLosses[i]
            totalStyleLoss += loss_i(tf_i, sf_i)
        totalStyleLoss = totalStyleLoss * self.style_weight

        loss = totalStyleLoss + totalContentLoss
        return loss, totalStyleLoss, totalContentLoss


class styleLoss(nn.Module):
    def forward(self,input,target):
        ib,ic,ih,iw = input.size()
        iF = input.view(ib,ic,-1)
        iMean = torch.mean(iF,dim=2)
        iCov = GramMatrix()(input)

        tb,tc,th,tw = target.size()
        tF = target.view(tb,tc,-1)
        tMean = torch.mean(tF,dim=2)
        tCov = GramMatrix()(target)

        loss = nn.MSELoss(size_average=False)(iMean,tMean) + nn.MSELoss(size_average=False)(iCov,tCov)
        return loss/tb

class GramMatrix(nn.Module):
    def forward(self,input):
        b, c, h, w = input.size()
        f = input.view(b,c,h*w) # bxcx(hxw)
        # torch.bmm(batch1, batch2, out=None)   #
        # batch1: bxmxp, batch2: bxpxn -> bxmxn #
        G = torch.bmm(f,f.transpose(1,2)) # f: bxcx(hxw), f.transpose: bx(hxw)xc -> bxcxc
        return G.div_(c*h*w)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from libs.Loader import Dataset
from libs.models import encoder4, decoder4
from libs.Criterion import LossCriterion



class Loss_sensitivity:
    def __init__(self, vgg, dec, matrix, style_layers, content_layers, style_weight, content_weight, device):
        self.vgg = vgg.to(device)
        self.dec = dec.to(device)
        self.matrix = matrix.to(device)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.criterion = LossCriterion(style_layers, content_layers, style_weight, content_weight)
        self.device = device

    def add_noise(self, matrix, sigma):
        noise = torch.randn_like(matrix) * sigma
        return matrix + noise

    def forward(self, contentV, styleV):
        with torch.no_grad():
            sF = self.vgg(styleV)
            cF = self.vgg(contentV)
        return sF, cF

    def run_experiment(self, contentV, styleV, sigmas):
        sigma_values = []
        loss_values = []

        sF, cF = self.forward(contentV, styleV)

        for sigma in sigmas:
            
            transformed_features, transmatrix = self.matrix(cF[self.style_layers[0]], sF[self.style_layers[0]])
            
            # Add noise to the transformation matrix
            noisy_transmatrix = self.add_noise(transmatrix, sigma)
            
            # Apply the noisy transformation matrix
            b, c, h, w = transformed_features.size()
            compressed_features = self.matrix.compress(transformed_features)
            noisy_transfeature = torch.bmm(noisy_transmatrix, compressed_features.view(b, self.matrix.matrixSize, -1))
            noisy_transfeature = noisy_transfeature.view(b, self.matrix.matrixSize, h, w)
            noisy_transfeature = self.matrix.unzip(noisy_transfeature)
            
            noisy_transfer = self.dec(noisy_transfeature).clamp(0, 1)

            tF = self.vgg(noisy_transfer)
            total_loss, _, _ = self.criterion(tF, sF, cF)

            sigma_values.append(sigma)
            loss_values.append(total_loss.item())
            print(f"Sigma: {sigma}, Total Loss: {total_loss.item()}")

        return sigma_values, loss_values
    
    def run_scaling_vector_experiment(self, contentV, styleV, scale_factors, vector_magnitudes):
        scale_values = []
        vector_values = []
        loss_values = []

        sF, cF = self.forward(contentV, styleV)

        # Get the original transformation matrix
        _, transmatrix = self.matrix(cF[self.style_layers[0]], sF[self.style_layers[0]])

        for scale in scale_factors:
            for vector_mag in vector_magnitudes:
                # Scale the transformation matrix
                scaled_matrix = transmatrix * scale

                # Create a simple vector and add it to each row of the matrix
                vector = torch.ones_like(scaled_matrix[0, 0]) * vector_mag
                modified_matrix = scaled_matrix + vector.unsqueeze(1)

                # Apply the modified transformation
                b, c, h, w = cF[self.style_layers[0]].size()
                compressed_features = self.matrix.compress(cF[self.style_layers[0]])
                modified_features = torch.bmm(modified_matrix, compressed_features.view(b, self.matrix.matrixSize, -1))
                modified_features = modified_features.view(b, self.matrix.matrixSize, h, w)
                modified_features = self.matrix.unzip(modified_features)

                modified_transfer = self.dec(modified_features).clamp(0, 1)

                tF = self.vgg(modified_transfer)
                total_loss, _, _ = self.criterion(tF, sF, cF)

                scale_values.append(scale)
                vector_values.append(vector_mag)
                loss_values.append(total_loss.item())
                print(f"Scale: {scale}, Vector Magnitude: {vector_mag}, Total Loss: {total_loss.item()}")

        return scale_values, vector_values, loss_values
    
    def run_dimension_dropping_experiment(self, contentV, styleV):
        sF, cF = self.forward(contentV, styleV)
        
        # Get the original features
        content_features = cF[self.style_layers[0]]
        style_features = sF[self.style_layers[0]]
        
        b, c, h, w = content_features.size()
        
        content_loss_impacts = []
        style_loss_impacts = []
        
        # Calculate baseline loss
        _, transmatrix = self.matrix(content_features, style_features)
        compressed_features = self.matrix.compress(content_features)
        modified_features = torch.bmm(transmatrix, compressed_features.view(b, self.matrix.matrixSize, -1))
        modified_features = modified_features.view(b, self.matrix.matrixSize, h, w)
        modified_features = self.matrix.unzip(modified_features)
        stylized_image = self.dec(modified_features).clamp(0, 1)
        tF = self.vgg(stylized_image)
        baseline_loss, _, _ = self.criterion(tF, sF, cF)
        
        # Analyze content dimensions
        for dim in range(c):
            dropped_content = content_features.clone()
            dropped_content[:, dim, :, :] = 0
            
            # Apply the transformation
            _, transmatrix = self.matrix(dropped_content, style_features)
            compressed_features = self.matrix.compress(dropped_content)
            modified_features = torch.bmm(transmatrix, compressed_features.view(b, self.matrix.matrixSize, -1))
            modified_features = modified_features.view(b, self.matrix.matrixSize, h, w)
            modified_features = self.matrix.unzip(modified_features)
            
            # Generate the stylized image and calculate loss
            stylized_image = self.dec(modified_features).clamp(0, 1)
            tF = self.vgg(stylized_image)
            total_loss, _, _ = self.criterion(tF, sF, cF)
            
            content_loss_impacts.append(total_loss.item() - baseline_loss.item())
        
        # Analyze style dimensions
        for dim in range(c):
            dropped_style = style_features.clone()
            dropped_style[:, dim, :, :] = 0
            
            # Apply the transformation
            _, transmatrix = self.matrix(content_features, dropped_style)
            compressed_features = self.matrix.compress(content_features)
            modified_features = torch.bmm(transmatrix, compressed_features.view(b, self.matrix.matrixSize, -1))
            modified_features = modified_features.view(b, self.matrix.matrixSize, h, w)
            modified_features = self.matrix.unzip(modified_features)
            
            # Generate the stylized image and calculate loss
            stylized_image = self.dec(modified_features).clamp(0, 1)
            tF = self.vgg(stylized_image)
            total_loss, _, _ = self.criterion(tF, sF, cF)
            
            style_loss_impacts.append(total_loss.item() - baseline_loss.item())
        
        return content_loss_impacts, style_loss_impacts
    
    def run_pca_experiment(self, contentV, styleV, n_components=50):
        sF, cF = self.forward(contentV, styleV)
        
        # Get the original features
        content_features = cF[self.style_layers[0]]
        style_features = sF[self.style_layers[0]]
        
        b, c, h, w = content_features.size()
        
        # Reshape features for PCA
        content_features_flat = content_features.view(b, c, -1).permute(0, 2, 1).reshape(-1, c)
        style_features_flat = style_features.view(b, c, -1).permute(0, 2, 1).reshape(-1, c)
        
        # Apply PCA
        content_pca = PCA(n_components=n_components)
        style_pca = PCA(n_components=n_components)
        
        content_pca.fit(content_features_flat.cpu().numpy())
        style_pca.fit(style_features_flat.cpu().numpy())
        
        # Calculate baseline loss
        _, transmatrix = self.matrix(content_features, style_features)
        compressed_features = self.matrix.compress(content_features)
        modified_features = torch.bmm(transmatrix, compressed_features.view(b, self.matrix.matrixSize, -1))
        modified_features = modified_features.view(b, self.matrix.matrixSize, h, w)
        modified_features = self.matrix.unzip(modified_features)
        stylized_image = self.dec(modified_features).clamp(0, 1)
        tF = self.vgg(stylized_image)
        baseline_loss, _, _ = self.criterion(tF, sF, cF)
        
        content_pca_loss_impacts = []
        style_pca_loss_impacts = []
        
        # Analyze PCA components for content
        for i in range(n_components):
            pca_component = torch.from_numpy(content_pca.components_[i]).float().to(self.device)
            content_flat = content_features.view(b, c, -1)
            projection = torch.matmul(content_flat.permute(0, 2, 1), pca_component.unsqueeze(1)).permute(0, 2, 1)
            dropped_content_flat = content_flat - projection * pca_component.unsqueeze(0).unsqueeze(-1)
            dropped_content = dropped_content_flat.view(b, c, h, w)
            
            # Apply the transformation
            _, transmatrix = self.matrix(dropped_content, style_features)
            compressed_features = self.matrix.compress(dropped_content)
            modified_features = torch.bmm(transmatrix, compressed_features.view(b, self.matrix.matrixSize, -1))
            modified_features = modified_features.view(b, self.matrix.matrixSize, h, w)
            modified_features = self.matrix.unzip(modified_features)
            
            # Generate the stylized image and calculate loss
            stylized_image = self.dec(modified_features).clamp(0, 1)
            tF = self.vgg(stylized_image)
            total_loss, _, _ = self.criterion(tF, sF, cF)
            
            content_pca_loss_impacts.append(total_loss.item() - baseline_loss.item())
        
        # Analyze PCA components for style (similar changes as content)
        for i in range(n_components):
            pca_component = torch.from_numpy(style_pca.components_[i]).float().to(self.device)
            style_flat = style_features.view(b, c, -1)
            projection = torch.matmul(style_flat.permute(0, 2, 1), pca_component.unsqueeze(1)).permute(0, 2, 1)
            dropped_style_flat = style_flat - projection * pca_component.unsqueeze(0).unsqueeze(-1)
            dropped_style = dropped_style_flat.view(b, c, h, w)
            
            # Apply the transformation
            _, transmatrix = self.matrix(content_features, dropped_style)
            compressed_features = self.matrix.compress(content_features)
            modified_features = torch.bmm(transmatrix, compressed_features.view(b, self.matrix.matrixSize, -1))
            modified_features = modified_features.view(b, self.matrix.matrixSize, h, w)
            modified_features = self.matrix.unzip(modified_features)
            
            # Generate the stylized image and calculate loss
            stylized_image = self.dec(modified_features).clamp(0, 1)
            tF = self.vgg(stylized_image)
            total_loss, _, _ = self.criterion(tF, sF, cF)
            
            style_pca_loss_impacts.append(total_loss.item() - baseline_loss.item())
        
        return content_pca_loss_impacts, style_pca_loss_impacts, content_pca, style_pca

In [ ]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load models
    vgg = encoder4()
    dec = decoder4()
    matrix = MulLayer('r41')
    
    vgg_dir = 'models/vgg_r41.pth'
    decoder_dir = "models/dec_r41.pth"
    
    vgg.load_state_dict(torch.load(vgg_dir))
    dec.load_state_dict(torch.load(decoder_dir))
    
    style_layers = ['r41']
    content_layers = ['r41']
    style_weight = 1e5
    content_weight = 1.0
    
    class Options:
        def __init__(self):
            self.contentPath = "data/content/"
            self.stylePath = "data/style/"
            self.loadSize = 256
            self.fineSize = 256

    opt = Options()
    
    loss_sensitivity = Loss_sensitivity(vgg, dec, matrix, style_layers, content_layers, style_weight, content_weight, device)
    
    # Load a single content and style image for the experiment
    content_dataset = Dataset(opt.contentPath, opt.loadSize, opt.fineSize)
    style_dataset = Dataset(opt.stylePath, opt.loadSize, opt.fineSize)
    
    contentV, _ = content_dataset[0]
    styleV, _ = style_dataset[0]
    
    contentV = contentV.unsqueeze(0).to(device)
    styleV = styleV.unsqueeze(0).to(device)
    
    # Run experiments on sigma
    sigmas = np.linspace(0, 0.2, 10)
    sigma_values, loss_values = loss_sensitivity.run_experiment(contentV, styleV, sigmas)
    
    # Plot the results
    plt.figure(figsize=(10, 6))
    plt.plot(sigma_values, loss_values, '-o')
    plt.xlabel('Sigma (Noise Level)')
    plt.ylabel('Total Loss')
    plt.title('Noise Sensitivity of Transformation Matrix')
    plt.grid(True)
    plt.savefig('noise_sensitivity_plot.png')
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load models
    vgg = encoder4()
    dec = decoder4()
    matrix = MulLayer('r41')
    
    vgg_dir = 'models/vgg_r41.pth'
    decoder_dir = "models/dec_r41.pth"
    
    vgg.load_state_dict(torch.load(vgg_dir))
    dec.load_state_dict(torch.load(decoder_dir))
    
    style_layers = ['r41']
    content_layers = ['r41']
    style_weight = 1e5
    content_weight = 1.0
    
    class Options:
        def __init__(self):
            self.contentPath = "data/content/"
            self.stylePath = "data/style/"
            self.loadSize = 256
            self.fineSize = 256

    opt = Options()
    
    loss_sensitivity = Loss_sensitivity(vgg, dec, matrix, style_layers, content_layers, style_weight, content_weight, device)
    
    # Load a single content and style image for the experiment
    content_dataset = Dataset(opt.contentPath, opt.loadSize, opt.fineSize)
    style_dataset = Dataset(opt.stylePath, opt.loadSize, opt.fineSize)
    
    contentV, _ = content_dataset[0]
    styleV, _ = style_dataset[0]
    
    contentV = contentV.unsqueeze(0).to(device)
    styleV = styleV.unsqueeze(0).to(device)
    
    # Run scaling and vector addition experiment
    scale_factors = np.linspace(0.5, 1.5, 5)
    vector_magnitudes = np.linspace(0, 0.1, 5)
    scale_values, vector_values, loss_values = loss_sensitivity.run_scaling_vector_experiment(contentV, styleV, scale_factors, vector_magnitudes)
    
    # Plot the results
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(scale_values, vector_values, loss_values, c=loss_values, cmap='viridis')
    ax.set_xlabel('Scale Factor')
    ax.set_ylabel('Vector Magnitude')
    ax.set_zlabel('Total Loss')
    ax.set_title('Loss Sensitivity to Scaling and Vector Addition')
    fig.colorbar(scatter, label='Total Loss')
    plt.savefig('scaling_vector_sensitivity_plot.png')
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load models
    vgg = encoder4()
    dec = decoder4()
    matrix = MulLayer('r41')
    
    vgg_dir = 'models/vgg_r41.pth'
    decoder_dir = "models/dec_r41.pth"
    
    vgg.load_state_dict(torch.load(vgg_dir))
    dec.load_state_dict(torch.load(decoder_dir))
    
    style_layers = ['r41']
    content_layers = ['r41']
    style_weight = 1e5
    content_weight = 1.0
    
    class Options:
        def __init__(self):
            self.contentPath = "data/content/"
            self.stylePath = "data/style/"
            self.loadSize = 256
            self.fineSize = 256

    opt = Options()
    
    loss_sensitivity = Loss_sensitivity(vgg, dec, matrix, style_layers, content_layers, style_weight, content_weight, device)
    
    # Load a single content and style image for the experiment
    content_dataset = Dataset(opt.contentPath, opt.loadSize, opt.fineSize)
    style_dataset = Dataset(opt.stylePath, opt.loadSize, opt.fineSize)
    
    contentV, _ = content_dataset[0]
    styleV, _ = style_dataset[0]
    
    contentV = contentV.unsqueeze(0).to(device)
    styleV = styleV.unsqueeze(0).to(device)
    
    # Run dimension dropping experiment
    content_loss_impacts, style_loss_impacts = loss_sensitivity.run_dimension_dropping_experiment(contentV, styleV)
    
    # Plot the results
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 16))
    
    ax1.plot(range(len(content_loss_impacts)), content_loss_impacts)
    ax1.set_xlabel('Content Feature Dimension')
    ax1.set_ylabel('Change in Total Loss')
    ax1.set_title('Impact of Dropping Individual Content Feature Dimensions on Total Loss')
    ax1.grid(True)
    
    ax2.plot(range(len(style_loss_impacts)), style_loss_impacts)
    ax2.set_xlabel('Style Feature Dimension')
    ax2.set_ylabel('Change in Total Loss')
    ax2.set_title('Impact of Dropping Individual Style Feature Dimensions on Total Loss')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('dimension_dropping_plot.png')
    plt.show()

if __name__ == "__main__":
    main()



In [ ]:
def main():
    # ... (previous setup code remains unchanged) ...
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Load models
    vgg = encoder4()
    dec = decoder4()
    matrix = MulLayer('r41')
    
    vgg_dir = 'models/vgg_r41.pth'
    decoder_dir = "models/dec_r41.pth"
    
    vgg.load_state_dict(torch.load(vgg_dir))
    dec.load_state_dict(torch.load(decoder_dir))
    
    style_layers = ['r41']
    content_layers = ['r41']
    style_weight = 1e5
    content_weight = 1.0
    
    class Options:
        def __init__(self):
            self.contentPath = "data/content/"
            self.stylePath = "data/style/"
            self.loadSize = 256
            self.fineSize = 256

    opt = Options()
    loss_sensitivity = Loss_sensitivity(vgg, dec, matrix, style_layers, content_layers, style_weight, content_weight, device)
    
    # Load a single content and style image for the experiment
    content_dataset = Dataset(opt.contentPath, opt.loadSize, opt.fineSize)
    style_dataset = Dataset(opt.stylePath, opt.loadSize, opt.fineSize)
    
    contentV, _ = content_dataset[0]
    styleV, _ = style_dataset[0]
    
    contentV = contentV.unsqueeze(0).to(device)
    styleV = styleV.unsqueeze(0).to(device)
    
    # Run dimension dropping experiment
    content_loss_impacts, style_loss_impacts = loss_sensitivity.run_dimension_dropping_experiment(contentV, styleV)
    
    # Run PCA experiment
    n_components = 50
    content_pca_loss_impacts, style_pca_loss_impacts, content_pca, style_pca = loss_sensitivity.run_pca_experiment(contentV, styleV, n_components)
    
    # Plot the results
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 20))
    
    ax1.plot(range(len(content_loss_impacts)), content_loss_impacts)
    ax1.set_xlabel('Content Feature Dimension')
    ax1.set_ylabel('Change in Total Loss')
    ax1.set_title('Impact of Dropping Individual Content Feature Dimensions')
    ax1.grid(True)
    
    ax2.plot(range(len(style_loss_impacts)), style_loss_impacts)
    ax2.set_xlabel('Style Feature Dimension')
    ax2.set_ylabel('Change in Total Loss')
    ax2.set_title('Impact of Dropping Individual Style Feature Dimensions')
    ax2.grid(True)
    
    ax3.plot(range(n_components), content_pca_loss_impacts)
    ax3.set_xlabel('Content PCA Component')
    ax3.set_ylabel('Change in Total Loss')
    ax3.set_title('Impact of Dropping Content PCA Components')
    ax3.grid(True)
    
    ax4.plot(range(n_components), style_pca_loss_impacts)
    ax4.set_xlabel('Style PCA Component')
    ax4.set_ylabel('Change in Total Loss')
    ax4.set_title('Impact of Dropping Style PCA Components')
    ax4.grid(True)
    
    plt.tight_layout()
    plt.savefig('dimension_and_pca_analysis_plot.png')
    plt.show()
    
    # Plot explained variance ratio
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    ax1.plot(range(1, n_components + 1), content_pca.explained_variance_ratio_)
    ax1.set_xlabel('Number of Components')
    ax1.set_ylabel('Explained Variance Ratio')
    ax1.set_title('Explained Variance Ratio of Content PCA Components')
    ax1.grid(True)
    
    ax2.plot(range(1, n_components + 1), style_pca.explained_variance_ratio_)
    ax2.set_xlabel('Number of Components')
    ax2.set_ylabel('Explained Variance Ratio')
    ax2.set_title('Explained Variance Ratio of Style PCA Components')
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('pca_explained_variance_plot.png')
    plt.show()

if __name__ == "__main__":
    main()